# Projet 8 - Etude de l'impact de l'algorithme Mask R-CNN sur le modèle du projet 7

## Initialisation

#### drive Google

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Librairies

In [0]:
import os
import sys
import math
# import pandas as pd
import numpy as np
import skimage.io
import skimage.transform
# from skimage.transform import rescale, resize
import matplotlib
import matplotlib.pyplot as plt

# Import joblib functions for save and load
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils.multiclass import unique_labels

# Root directory of the project
# ROOT_DIR = os.path.abspath("../")
ROOT_DIR = r"/content/gdrive/My Drive/Colab Notebooks/P8 - Mask R_CNN/"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
IMAGE_M_DIR = os.path.join(ROOT_DIR, "imagesM")
IMAGE_MB_DIR = os.path.join(ROOT_DIR, "imagesMB")
IMAGE_B_DIR = os.path.join(ROOT_DIR, "imagesB")
IMAGE_LOG_DIR = os.path.join(ROOT_DIR, "logs")
IMAGE_LOG0_DIR = os.path.join(IMAGE_LOG_DIR, "nodog")
IMAGE_LOG1_DIR = os.path.join(IMAGE_LOG_DIR, "noanimal")

In [0]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model, Sequential, load_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
  
import h5py

In [0]:
main_dirs = [IMAGE_M_DIR, IMAGE_MB_DIR, IMAGE_B_DIR,
             IMAGE_LOG_DIR, IMAGE_LOG0_DIR, IMAGE_LOG1_DIR]
for main_dir in main_dirs:
  try:
    os.mkdir(main_dir)
  except FileExistsError:
    pass

### Configurations

In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [0]:
INPUT_SIZE = 224

### Create Model and Load Trained Weights

In [0]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








### Class names

In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [0]:
DOG_CLASS = 17
ANIMAL_CLASSES = range(16, 25)

## Mask R-CNN functions

In [0]:
def select_mask(image, r, file_name):
    found_one = False
    
    for i in range(len(r['scores'])):
        if r['class_ids'][i] == DOG_CLASS:
            return r['masks'][:,:,i:i+1], r['rois'][i:i+1][0]
        if (r['class_ids'][i] in ANIMAL_CLASSES) and not found_one:
            mask, box = r['masks'][:,:,i:i+1], r['rois'][i:i+1][0]
            found_one = True
    
    if found_one:
      # print("Masques non chien trouvés pour {}".format(file_name))
      visualize.save_instances(image, r['rois'][0:1], r['masks'][:,:,0:1], r['class_ids'][0:1], 
                            class_names, os.path.join(IMAGE_LOG0_DIR, file_name) , r['scores'][0:1])
      return mask, box
    else:      
      # print("Aucun masque trouvé pour {}".format(file_name))
      visualize.save_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, os.path.join(IMAGE_LOG1_DIR, file_name), r['scores'])
      return [],[]

In [0]:
def apply_mask(dir_name, file_name):
    """Apply Mask-RCNN to an image, and return image with detected mask applied
    file_name: name of the file in IMAGE_DIR
    
    returns an image array"""
    
    image = skimage.io.imread(os.path.join(IMAGE_DIR, dir_name, file_name))

    # Run detection
    results = model.detect([image], verbose=0)[0]
    
    # Select the most relevant mask
    mask, box_coord = select_mask(image, results, file_name)
    
    if len(mask) == 0:
      # If no mask selected, return original image and save log with result of Mask R-CNN
      imgM = imgMB = imgB = image           
      # box_dim = image.shape
    else:
      x0, y0, x1, y1 = box_coord[0], box_coord[1], box_coord[2], box_coord[3] 
      # If a mask is selected, apply mask to original image
      imgM = image * mask
      # cropping sur bounding box
      imgB = image[x0:x1, y0:y1]   
      imgMB = imgM[x0:x1, y0:y1]    
      # box_dim = (x1-x0, y1-y0)

    # Save image with Mask applied  
    skimage.io.imsave(fname=os.path.join(IMAGE_M_DIR, dir_name, file_name), arr= imgM, check_contrast=False, quality=100)
    # Save image with Mask and Box
    skimage.io.imsave(fname=os.path.join(IMAGE_MB_DIR, dir_name, file_name), arr= imgMB, check_contrast=False, quality=100)
    # Save image with Box
    skimage.io.imsave(fname=os.path.join(IMAGE_B_DIR, dir_name, file_name), arr= imgB, check_contrast=False, quality=100)
  
    # return img

In [0]:
dirs = sorted(os.listdir(IMAGE_DIR))
for dir_name in dirs
  print(dir_name)
  for main_dir in [IMAGE_M_DIR, IMAGE_MB_DIR, IMAGE_B_DIR]:
    try:
      os.mkdir(os.path.join(main_dir, dir_name))
    except FileExistsError:
      pass
  files = sorted(os.listdir(os.path.join(IMAGE_DIR, dir_name)))
  for file in files:
    apply_mask(dir_name, file)

n02105855-Shetland_sheepdog
n02106030-collie
n02106166-Border_collie
n02106382-Bouvier_des_Flandres
n02106550-Rottweiler
n02106662-German_shepherd
n02107142-Doberman
n02107312-miniature_pinscher
n02107574-Greater_Swiss_Mountain_dog
n02107683-Bernese_mountain_dog
n02107908-Appenzeller
n02108000-EntleBucher
n02108089-boxer
n02108422-bull_mastiff
n02108551-Tibetan_mastiff
n02108915-French_bulldog
n02109047-Great_Dane
n02109525-Saint_Bernard
n02109961-Eskimo_dog
n02110063-malamute
n02110185-Siberian_husky
n02110627-affenpinscher
n02110806-basenji
n02110958-pug
n02111129-Leonberg
n02111277-Newfoundland
n02111500-Great_Pyrenees
n02111889-Samoyed
n02112018-Pomeranian
n02112137-chow
n02112350-keeshond
n02112706-Brabancon_griffon
n02113023-Pembroke
n02113186-Cardigan
n02113624-toy_poodle
n02113712-miniature_poodle
n02113799-standard_poodle
n02113978-Mexican_hairless
n02115641-dingo
n02115913-dhole
n02116738-African_hunting_dog
